In [ ]:
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
from sklearn.utils import shuffle
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

data = pd.read_excel(r"/kaggle/input/new-data-ji-ji/jiji small126.xlsx",engine="openpyxl")
X = data.iloc[:, :5].values
y = data.iloc[:, -1].values

test_losses = []
temp = 100000.0
temp= torch.tensor(temp, dtype=torch.float32)
test_losses.append(temp)
print(torch.sqrt(sum(test_losses)))

In [ ]:
input_size = X[0].shape[0]
output_size = 1
learning_rate = 1e-3
num_epochs = 1000
save_rate = 0.98
jiaocha_numbers = 10
Linear_size1 = 96
Linear_size2 = 64
Linear_flag=0
num_while=0

In [ ]:
# 定义模型
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.input_size = input_size

        self.fnn1 = nn.Sequential(
            nn.Linear(input_size,Linear_size1),
            nn.ELU(),
            nn.BatchNorm1d(Linear_size1),
            nn.Linear(Linear_size1,1),
        )

    def forward(self, x):
        out = self.fnn1(x)
        return out

In [ ]:
while (torch.sqrt(sum(test_losses)/len(y)) > 0.235):
    num_while +=1
    test_losses = []
    if Linear_size1== 64 :
        Linear_size1 = 97
    Linear_size1 -= 1
    
    # 定义模型
    class MLP(nn.Module):
        def __init__(self, input_size):
            super(MLP, self).__init__()
            self.input_size = input_size

            self.fnn1 = nn.Sequential(
                nn.Linear(input_size,Linear_size1),
                nn.ELU(),
                nn.BatchNorm1d(Linear_size1),
                nn.Linear(Linear_size1,1),
            )

        def forward(self, x):
            out = self.fnn1(x)
            return out
    
    kf = KFold(n_splits=jiaocha_numbers ,shuffle=True)
    results_df = pd.DataFrame()
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        model = MLP(input_size)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=0.0001)
        lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epochs,last_epoch=-1)
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        scaler = MinMaxScaler()
        
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
        
        train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        batch_size = X_train.shape[0]
        train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        
        losses = []
        
#        bar = tqdm(range(num_epochs))
#        bar.set_description("[ Training ]")
        bar = range(num_epochs)
        best = 100
        for idx,epoch in enumerate(bar):
            for i, (inputs, labels) in enumerate(train_loader):
                labels = labels 
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                if loss.item()<best and idx>num_epochs*save_rate:
                    best = loss.item()
                    torch.save(model.state_dict(), f'./model_fold_{fold + 1}.ckpt')
#                    print(f"loss:{loss.item()} saving...")

                loss.backward()
                optimizer.step()

            losses.append(loss.item())
            lr_scheduler.step()
#            if (epoch+1) % 100 == 0 :
#                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.10f}')
#            bar.set_postfix({ 'Loss': f'{loss.item():.10f}','lr':f"{lr_scheduler.get_lr()[0]:.6f}"})

        model.load_state_dict(torch.load(f'./model_fold_{fold + 1}.ckpt'))
        model.eval()
        
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
            y_pred = test_outputs.to('cpu')
            test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
            rmse = torch.sqrt(test_loss)  # 计算均方根误差
            y_out = (y_pred - y_test_tensor.numpy().reshape(len(y_test_tensor),1))/y_test_tensor.numpy().reshape(len(y_test_tensor),1)
            y_out = y_out.abs() < 0.1
            accuracy = y_out.sum()/len(y_out)
#            print(f'Test Loss: {test_loss:.10f}, Test RMSE: {rmse:.10f},Accuracy: {accuracy:.10f}')
            
        X_test = scaler.inverse_transform(X_test)
        fold_df = pd.DataFrame(X_test, columns = [str(i) for i in range(1, X[0].shape[0]+1)])
        fold_df['Test Labels'] = y_test
        fold_df['Test Outputs'] = y_pred.numpy()
        fold_df['Fold'] = fold + 1
        results_df = pd.concat([results_df, fold_df], ignore_index=True)
        test_losses.append(test_loss*len(y_test))
        
    print(f'The {num_while} times of test: The RMSE of this test:{torch.sqrt(sum(test_losses)/len(y)):.10f} and Linear_size1 = {Linear_size1}')
        
results_df.to_excel('test_data_and_outputs.xlsx', index=False)
print(f'Times:{num_while} and Linear_size1 = {Linear_size1}')